In [1]:
import os 
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero
dir_ = '../../../data/'
group_dir_ = os.path.join(dir_, 'groups/high')

In [2]:
file_name = 'normalized_to_rating_filter_track_5_user_100.csv'
all_user = pd.read_pickle(os.path.join(dir_, file_name[:-3] + 'pkl'))
df = pd.read_pickle(os.path.join(dir_, file_name[:-3] + 'pkl'))
svd = pd.read_pickle(os.path.join(dir_, 'prediction_svd_top_N_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])

file_name = 'normalized_popularity_filter_track_5_user_100.pkl'
pop_count = pd.read_pickle(os.path.join(dir_, file_name))

num_user = len(svd['uid'].unique())
num_user

953

In [3]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8', '9', '10']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)

In [4]:
pop_dict = {}
for i in all_user['tid'].unique():
    pop_dict[i] = len(all_user[all_user['tid']==i])/num_user

In [5]:
svd['count'] = 1
svd[:5]

,uid,tid,rating,count
0,0,9,2.243909,1
1,0,15,2.160610,1
2,0,22,2.145235,1
3,0,28,2.204445,1
4,0,33,2.166677,1


In [6]:
pop_count = pop_count.sort_values(by=['count'],  ascending=False)
pop_count[:10]

,tid,count,rating
70,70,17557,2980.75
1102,1102,6926,1484.25
83,83,6571,1796.75
13496,13496,6299,1669.75
952,952,6234,1527.50
2519,2519,6001,1772.25
4008,4008,5676,1008.25
5521,5521,5658,1727.25
3900,3900,5627,1475.50
2241,2241,5571,1183.75


In [7]:
# min_max
max_rating = pop_count.iloc[0]['count']
pop_normalized = pop_count.copy()
pop_normalized = pop_normalized[['tid', 'count']]
pop_normalized['rating'] = pop_normalized['count']
pop_normalized['rating'] /= max_rating
pop_normalized['rating'] *= 5
pop_normalized[:10]

,tid,count,rating
70,70,17557,5.000000
1102,1102,6926,1.972433
83,83,6571,1.871333
13496,13496,6299,1.793871
952,952,6234,1.775360
2519,2519,6001,1.709005
4008,4008,5676,1.616449
5521,5521,5658,1.611323
3900,3900,5627,1.602495
2241,2241,5571,1.586547


In [8]:
lambdas = []
for i in range(11):
    lambdas.append(i/10)
lambdas

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [9]:
NOVs = []
COVs = []
HitNOVs = []
HitCOVs = []
for lambda_ in tqdm(lambdas):
    top_n_size = 50
    top_n_lists_g = []
    top_n_items_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []
        top_n_items = pd.DataFrame()

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = pop_normalized.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
            
            if len(top_n_items) == 0:
                top_n_items = rating_table
            else:
                top_n_items = top_n_items.set_index('tid').add(rating_table.set_index('tid'), fill_value=0).reset_index()
                
        top_n_lists_g.append(top_n_lists)   
        top_n_items_g.append(top_n_items)
        
    COV = []    
    NOV = []
    HitNOV = []
    HitCOV = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]
        top_n_items = top_n_items_g[i]

        nov = 0
        hitNov = 0
        hits = np.zeros(len(df['tid'].unique()), dtype=bool)
        for j in (range(len(groups))):
            group = groups[j]
            top_n_list = top_n_lists[j]
            
            for k in range(len(group)):
                for l in range(top_n_size):
                    uid = group[k]
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    nov += (1-pop_dict[tid])
                    if len(t) > 0:
                        hits[int(tid)] = True
                        hitNov += (1-pop_dict[tid])
        NOV.append(nov/top_n_size/(len(groups)*len(groups[0])))
        COV.append(len(top_n_items)/len(df['tid'].unique()))
        HitNOV.append(hitNov/top_n_size/(len(groups)*len(groups[0])))
        HitCOV.append(list(hits).count(True)/len(df['tid'].unique()))
    NOVs.append(NOV)
    COVs.append(COV)
    HitNOVs.append(HitNOV)
    HitCOVs.append(HitCOV)
    
for NOV in NOVs:
    print(NOV)
print()
for HitNOV in HitNOVs:
    print(HitNOV)
print()  
for COV in COVs:
    print(COV)
print()    
for HitCOV in HitCOVs:
    print(HitCOV)


[0.0799444145089471, 0.07233059270967535, 0.06466386848548465, 0.05835276514709897, 0.046645381721393264, 0.04236582584956134, 0.03892251875903296, 0.032873757983236794, 0.031628164769822706]
[0.0799444145089471, 0.07233059270967535, 0.06466386848548465, 0.05835276514709897, 0.046645381721393264, 0.04236582584956134, 0.03892251875903296, 0.032873757983236794, 0.031628164769822706]
[0.0799444145089471, 0.07233059270967535, 0.06466386848548465, 0.05835276514709897, 0.046645381721393264, 0.04236582584956134, 0.03892251875903296, 0.032873757983236794, 0.031628164769822706]
[0.0799444145089471, 0.07233059270967535, 0.06466386848548465, 0.05835276514709897, 0.046645381721393264, 0.04236582584956134, 0.03892251875903296, 0.032873757983236794, 0.031628164769822706]
[0.0799444145089471, 0.07233059270967535, 0.06466386848548465, 0.05835276514709897, 0.046645381721393264, 0.04236582584956134, 0.03892251875903296, 0.032873757983236794, 0.031628164769822706]
[0.0799444145089471, 0.0723305927096753

In [10]:
for NOV in NOVs:
    print(NOV)

[0.5490661070304452, 0.5490661070304448, 0.549066107030444, 0.5490661070304474, 0.5490661070304509, 0.5490661070304534, 0.5490661070304571, 0.5490661070304592, 0.5490661070304611]
[0.5619126977784035, 0.5620730449258746, 0.5632674361664973, 0.5647205705180557, 0.5635231489775633, 0.5638405036726247, 0.5632536775624354, 0.566195173137479, 0.5645830979094589]
[0.5791584315404101, 0.5819214472618307, 0.5824243441762956, 0.5821796665500755, 0.5830878099966769, 0.5835886673662177, 0.5828897820190576, 0.5857637868718845, 0.5853359162698256]
[0.6007639653390272, 0.6016912171645684, 0.6064339979013857, 0.6073838949127571, 0.6112182325391012, 0.6117879741804281, 0.6151081985388873, 0.6145174303369635, 0.6190905911157853]
[0.6127578499037312, 0.6309867573419423, 0.6373709688702605, 0.6446463409894783, 0.6482855168530769, 0.654697764634815, 0.6578091033281327, 0.6597724145971939, 0.6664040681249588]
[0.6279803852311792, 0.6485062936939423, 0.6671965022735414, 0.6801919863200109, 0.692204591405840